In [ ]:
task = "Mistake_Identification"

In [ ]:
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"

In [ ]:
model_to_be_trained = "mistral-7b-instruct"

In [ ]:
extra = '-zero-shot'

In [5]:
new_folder_name = f'{model_to_be_trained}-{task}{extra}'

In [ ]:
new_folder_name

### Connect to Drive

In [7]:
# prompt: connect to google drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


### Connect to Wandb

In [ ]:
import wandb
WANDB_API_KEY="YOUR_WANDB_API_KEY"
wandb.login(key=WANDB_API_KEY)
wandb_project = new_folder_name
wandb.init(project=wandb_project)

### Installation

In [9]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Mistral patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/141k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [11]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>

In [ ]:
from datasets import load_dataset, Dataset, DatasetDict
from sklearn.model_selection import train_test_split

alpaca_prompt = """### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Load dataset using Hugging Face Datasets
# the instruction was included with the jsonl file
# see the augmented fine-tuning code to check the instruction
dataset = load_dataset("json", data_files=f"/content/drive/~/Full_devset_with_mistral_instruction.jsonl")

# Function to format prompts
EOS_TOKEN = tokenizer.eos_token  # Ensure tokenizer is defined before this

def formatting_prompts_func(example):
    text = alpaca_prompt.format(example["instruction"], example["input"], "Evaluation: "+example["output"]) + EOS_TOKEN
    return {"text": text, "output": example["output"]}

# Apply formatting
formatted_dataset = dataset.map(formatting_prompts_func)

# Convert to Pandas DataFrame
df = formatted_dataset["train"].to_pandas()

# Perform stratified train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["output"], random_state=42)

# Convert back to Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df.reset_index(drop=True))
test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

# Create DatasetDict
split_dataset = DatasetDict({"train": train_dataset, "test": test_dataset})



Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2476 [00:00<?, ? examples/s]

### Dataset Analysis:

In [13]:
# Print class distributions to verify stratification
print("Train Class Distribution:\n", train_dataset.to_pandas()["output"].value_counts())
print("\nTest Class Distribution:\n", test_dataset.to_pandas()["output"].value_counts())

Train Class Distribution:
 output
Yes               1545
No                 296
To some extent     139
Name: count, dtype: int64

Test Class Distribution:
 output
Yes               387
No                 74
To some extent     35
Name: count, dtype: int64


In [14]:
# import matplotlib.pyplot as plt

# def plot_class_distribution(df, title):
#     counts = df["output"].value_counts()
#     plt.figure(figsize=(10, 5))
#     counts.plot(kind="bar")
#     plt.title(title)
#     plt.xlabel("Class")
#     plt.ylabel("Count")
#     plt.xticks(rotation=45)
#     plt.grid(True)
#     plt.tight_layout()
#     plt.show()

# plot_class_distribution(train_df, "Training Set Class Distribution")
# plot_class_distribution(test_df, "Test Set Class Distribution")

<a name="Inference"></a>
### Inference


In [15]:
inference_dataset = test_dataset
import time
start = time.time()

In [ ]:
# output folder
import os
new_folder_name = f'{model_to_be_trained}-{task}{extra}'
base_path = '/content/drive/~'  # Replace with your desired base path

new_folder_path = os.path.join(base_path, new_folder_name)

if not os.path.exists(new_folder_path):
  os.makedirs(new_folder_path)
  print(f"Folder '{new_folder_name}' created at '{new_folder_path}'")
else:
  print(f"Folder '{new_folder_name}' already exists at '{new_folder_path}'")


In [17]:
import torch
import pandas as pd
import re
from sklearn.metrics import f1_score, classification_report

# Ensure the model is in inference mode
FastLanguageModel.for_inference(model)

# Store results
results = []

for example in inference_dataset:
    # Tokenize input
    inputs = tokenizer([example["text"].split("\nEvaluation")[0]], return_tensors="pt").to("cuda")

    # Generate response
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        use_cache=True,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )

    # Decode and extract response
    response_text = tokenizer.batch_decode(outputs)[0]
    extracted_text = response_text.split("### Response:", 1)[-1].strip()

    # Extract mistake identification decision using regex
    # match = re.search(r"Evaluation:\s*(Yes \(and the answer is correct\)|No|Yes \(but the answer is incorrect\))", extracted_text, re.IGNORECASE)
    match = re.search(r"Evaluation:\s*(Yes|No|To some extent)", extracted_text, re.IGNORECASE)
    model_prediction = match.group(1) if match else "Unknown"

    # Store results
    results.append({
        "Instruction": example['instruction'],
        "Input": example['input'],
        "Model Output": extracted_text,
        "Extracted Prediction": model_prediction,
        "Ground Truth": example['output'],
        "Match": model_prediction.lower() == example['output'].lower()
    })

# Convert results to DataFrame
df = pd.DataFrame(results)

# Compute accuracy
accuracy = df["Match"].mean() * 100

# Convert labels to categorical format for F1-score computation
label_mapping = {
    "Yes": 0,
    "No": 1,
    "To some extent": 2
}


# Save DataFrame to CSV
df.to_csv(f"{new_folder_path}/{task}_results.csv", index=False)

# Print evaluation results
print(f"Accuracy: {accuracy:.2f}%")

# Display sample of results
df.head()

Accuracy: 70.16%


,Instruction,Input,Model Output,Extracted Prediction,Ground Truth,Match
0,Evaluate the tutor's response based on whether...,"Tutor: Hi, could you please provide a step-by-...",Evaluation: No. The tutor did not identify a m...,No,No,True
1,Evaluate the tutor's response based on whether...,"Tutor: Hi, could you please provide a step-by-...",Evaluation: Yes. The tutor has identified a mi...,Yes,Yes,True
2,Evaluate the tutor's response based on whether...,"Tutor: Hi, could you please provide a step-by-...",Evaluation: Yes. The tutor has identified a mi...,Yes,Yes,True
3,Evaluate the tutor's response based on whether...,"Tutor: Hi, could you please provide a step-by-...",Evaluation: Yes. The tutor has identified a mi...,Yes,Yes,True
4,Evaluate the tutor's response based on whether...,Tutor: Please have a look at the steps on the ...,Evaluation: Yes. The tutor has identified a mi...,Yes,Yes,True


In [18]:
time.time() - start

1938.7790281772614

In [19]:
import pandas as pd
from sklearn.metrics import classification_report

# Ensure both columns are strings and normalize case
df["Extracted Prediction"] = df["Extracted Prediction"].astype(str).str.lower().apply(lambda x: x.strip().strip("."))
df["Ground Truth"] = df["Ground Truth"].astype(str).str.lower().apply(lambda x: x.strip().strip("."))

# Compute classification report
report = classification_report(df["Ground Truth"], df["Extracted Prediction"], digits=4, zero_division=0)

# Print classification report
print(report)


                precision    recall  f1-score   support

            no     0.6111    0.5946    0.6027        74
to some extent     0.1013    0.2286    0.1404        35
           yes     0.8580    0.7649    0.8087       387

      accuracy                         0.7016       496
     macro avg     0.5234    0.5293    0.5173       496
  weighted avg     0.7677    0.7016    0.7308       496

